###LOAD RAW DATA

In [87]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, time, timedelta

# ============================
# 1. LOAD RAW DATA
# ============================

df_prod_raw = pd.read_csv("Production Data (Mar-August 2024).csv")
df_maint_raw = pd.read_csv("Maintenance downtime .csv")

# Strip whitespace from column names
df_prod_raw.columns = df_prod_raw.columns.str.strip()
df_maint_raw.columns = df_maint_raw.columns.str.strip()

print("Production columns:", df_prod_raw.columns.tolist())
print("Maintenance columns:", df_maint_raw.columns.tolist())

Production columns: ['CID', 'UID', 'Type', 'Production Date', 'Hours', 'Month', 'Month(Number)', 'Year', 'Cast', 'Slab', 'Grade', 'Reclass Code', 'Comments', 'Width', 'Widths', 'Thick', 'Thickess', 'MassIN', 'MassOut', 'Mass out tons', 'NextProcess']
Maintenance columns: ['Start', 'Hierachy', 'Decription', 'Duration', 'Day', 'Crew', 'Shifts', 'Time (Hours)', 'Reasponsible', 'Responsible', 'Date', 'Category', 'Delay Type', 'Area', 'Sub Area', 'Equipment']


###FILTER TO APRIL–JUNE 2024

In [88]:
# ============================
# 2. FILTER TO APRIL–JUNE 2024
# ============================

df_prod = df_prod_raw.copy()
df_prod["Production Date"] = pd.to_datetime(df_prod["Production Date"])

start = pd.Timestamp("2024-04-01 00:00:00")
end   = pd.Timestamp("2024-06-30 23:59:59")

df_prod = df_prod[
    (df_prod["Production Date"] >= start) &
    (df_prod["Production Date"] <= end)
].copy()

# Maintenance: use Date + Start
df_maint = df_maint_raw.copy()
df_maint["Date"] = pd.to_datetime(df_maint["Date"])
df_maint["Start"] = pd.to_datetime(df_maint["Start"], errors="coerce")

# Prefer the timestamp in Start; if null, use Date at midnight
df_maint["start_datetime"] = df_maint["Start"].combine_first(df_maint["Date"])

df_maint = df_maint[
    (df_maint["start_datetime"] >= start) &
    (df_maint["start_datetime"] <= end)
].copy()

/tmp/ipython-input-3817585346.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_prod["Production Date"] = pd.to_datetime(df_prod["Production Date"])
/tmp/ipython-input-3817585346.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_maint["Date"] = pd.to_datetime(df_maint["Date"])
/tmp/ipython-input-3817585346.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_maint["Start"] = pd.to_datetime(df_maint["Start"], errors="coerce")


In [89]:

# ============================
# 3. CLEAN SUB AREA -> EQUIPMENT NAME
# ============================

def clean_subarea(x):
    if pd.isna(x):
        return None
    x = str(x).strip()
    # Remove trailing "(number)" pattern, e.g. "(1)", "(14)", "(222)"
    x = re.sub(r"\(\d+\)$", "", x).strip()
    return x

df_maint["SubArea_Clean"] = df_maint["Sub Area"].apply(clean_subarea)


In [90]:

# ============================
# 4. BUILD dim_equipment FROM SubArea_Clean (NO CRANE)
# ============================

equip_series = (
    df_maint["SubArea_Clean"]
    .dropna()
    .astype(str)
    .str.strip()
)

unique_equipment = sorted(equip_series.unique())

# Exclude crane entirely
line_equipment_names = [
    e for e in unique_equipment
    if "CRANE" not in e.upper()
]

dim_equipment = pd.DataFrame({
    "equipment_name": line_equipment_names
})

dim_equipment["equipment_id"] = range(1, len(dim_equipment) + 1)
dim_equipment["process_order"] = np.nan          # will fill later
dim_equipment["section"] = None                  # 'ENTRY','CENTRE','EXIT'
dim_equipment["equipment_type"] = None           # 'Shear','Coil Car', etc.
dim_equipment["is_bottleneck_candidate"] = False
dim_equipment["is_active"] = True

# Reorder columns
dim_equipment = dim_equipment[[
    "equipment_id", "equipment_name", "process_order",
    "section", "equipment_type", "is_bottleneck_candidate", "is_active"
]]

print("\nInitial dim_equipment (before you fill process_order):")
dim_equipment.head()


Initial dim_equipment (before you fill process_order):


,equipment_id,equipment_name,process_order,section,equipment_type,is_bottleneck_candidate,is_active
0,1,CCTV Camera,NaN,None,None,False,True
1,2,Central Hyd System,NaN,None,None,False,True
2,3,Coil Prep Sattion,NaN,None,None,False,True
3,4,Common Equipment,NaN,None,None,False,True
4,5,Computer Room,NaN,None,None,False,True


In [91]:
unique_equipment_names_from_dim = dim_equipment['equipment_name'].unique()
print(unique_equipment_names_from_dim)

['CCTV Camera' 'Central Hyd System' 'Coil Prep Sattion' 'Common Equipment'
 'Computer Room' 'Crop Shear' 'Decoiler' 'Delivery Conveyor'
 'Entry & Exit Feed Table' 'Entry Guide Table'
 'Entry SnubberHold Down & Pressure Rolls' 'Exit Coil Car'
 'Farval Systems' 'First Conveyor' 'Flattener, Pinch & Deflator Rolls'
 'General' 'Lube System' 'Operation' 'Operations' 'Roll Shop'
 'Scale M65 (conveyor)' 'Second Conveyor' 'Services' 'Shutdown'
 'Temper Mill Unit']


In [92]:
# ============================
# 4.1 FILL PROCESS ORDER & BOTTLENECK FLAGS
# ============================

process_order_map = {
    "Coil Prep Sattion": 1,
    "Decoiler": 2,
    "Entry SnubberHold Down & Pressure Rolls": 3,
    "Entry Guide Table": 4,
    "Entry & Exit Feed Table": 5,
    "Flattener, Pinch & Deflator Rolls": 6,
    "Temper Mill Unit": 7,
    "Crop Shear": 8,
    "First Conveyor": 9,
    "Second Conveyor": 10,
    "Delivery Conveyor": 11,
    "Scale M65 (conveyor)": 12,
    "Exit Coil Car": 13,
}

bottleneck_map = {
    "Temper Mill Unit": True,
    "Exit Coil Car": True,
    "Decoiler": True,
    "Scale M65 (conveyor)": True,
}

# Mark non-line / utility equipment as inactive for the coil path
non_line = [
    "CCTV Camera",
    "Central Hyd System",
    "Common Equipment",
    "Computer Room",
    "Farval Systems",
    "General",
    "Lube System",
    "Operation",
    "Operations",
    "Roll Shop",
    "Services",
    "Shutdown",
]

# Apply mappings
dim_equipment["process_order"] = dim_equipment["equipment_name"].map(process_order_map)
dim_equipment["is_bottleneck_candidate"] = (
    dim_equipment["equipment_name"].map(bottleneck_map).fillna(False)
)

# Default all to active, then switch non-line to inactive
dim_equipment["is_active"] = True
dim_equipment.loc[
    dim_equipment["equipment_name"].isin(non_line),
    "is_active"
] = False

# Keep only active, ordered equipment for the coil path
line_equipment = (
    dim_equipment
    .query("is_active == True")
    .dropna(subset=["process_order"])
    .sort_values("process_order")
    .reset_index(drop=True)
)

print("\nLine equipment in process order (check this carefully):")
line_equipment[["equipment_id", "equipment_name", "process_order", "is_bottleneck_candidate"]]


Line equipment in process order (check this carefully):


/tmp/ipython-input-4144195053.py:47: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dim_equipment["equipment_name"].map(bottleneck_map).fillna(False)


,equipment_id,equipment_name,process_order,is_bottleneck_candidate
0,3,Coil Prep Sattion,1.0,False
1,7,Decoiler,2.0,True
2,11,Entry SnubberHold Down & Pressure Rolls,3.0,False
3,10,Entry Guide Table,4.0,False
4,9,Entry & Exit Feed Table,5.0,False
5,15,"Flattener, Pinch & Deflator Rolls",6.0,False
6,25,Temper Mill Unit,7.0,True
7,6,Crop Shear,8.0,False
8,14,First Conveyor,9.0,False
9,22,Second Conveyor,10.0,False


In [93]:
# ============================
# 5. BUILD fact_production_coil BASE
# ============================

prod = df_prod.copy()

# Thickness: prefer 'Thickess' if present, else 'Thick'
if "Thickess" in prod.columns:
    prod["thickness_mm"] = prod["Thickess"]
else:
    prod["thickness_mm"] = prod["Thick"]

# Basic geometry and mass
prod["width_mm"] = prod["Width"]
prod["mass_out_tons"] = prod["Mass out tons"]

# Date and coil ID
prod["production_date"] = prod["Production Date"].dt.date
prod["coil_id"] = prod["CID"].astype(str)

# Placeholder shift_code (synthetic rotation logic will overwrite this later)
prod["shift_code"] = "A"

# Base fact table
fact_production_coil = prod[[
    "coil_id",
    "production_date",
    "shift_code",
    "thickness_mm",
    "width_mm",
    "mass_out_tons",
    "Hours",
    "Grade",
    "NextProcess"
]].copy()

print("\nfact_production_coil sample:")
fact_production_coil.head()


fact_production_coil sample:


,coil_id,production_date,shift_code,thickness_mm,width_mm,mass_out_tons,Hours,Grade,NextProcess
126,7241079,2024-04-01,A,1.8,1211,23.40,00:38:51,AC1 37,13
127,7255091,2024-04-01,A,2.3,1200,21.60,00:57:57,RC3 1,13
128,7232058,2024-04-01,A,1.5,1236,22.56,00:25:08,AC2 31,13
129,7232058,2024-04-01,A,1.5,1236,0.84,00:22:22,AC2 31,13
130,7258061,2024-04-02,A,1.4,1232,21.94,00:18:53,AC2 31,13


In [94]:
# ============================
# 6. BUILD fact_maintenance_event BASE
# ============================

maint = df_maint.copy()

# --- 6.1 Duration hours ---
# Check for "Time (Hours)" first (correct after .str.strip())
if "Time (Hours)" in maint.columns:
    maint["duration_hours"] = pd.to_numeric(maint["Time (Hours)"], errors="coerce")

# If not found, fall back to "Duration"
elif "Duration" in maint.columns:
    maint["duration_hours"] = pd.to_numeric(maint["Duration"], errors="coerce")

# Last fallback (extremely rare)
else:
    raise ValueError("No valid duration column found in maintenance file.")

# Duration in minutes
maint["duration_min"] = maint["duration_hours"] * 60


# --- 6.2 Equipment name (from SubArea_Clean) ---
# Ensure the cleaned SubArea exists
if "SubArea_Clean" not in maint.columns:
    raise ValueError("SubArea_Clean column does not exist. Make sure cleaning step ran.")

maint["equipment_name"] = maint["SubArea_Clean"].astype(str).str.strip()


# --- 6.3 Build the fact table ---
required_cols = [
    "start_datetime",
    "duration_hours",
    "duration_min",
    "equipment_name",
    "Crew",
    "Shifts",
    "Category",
    "Delay Type",
    "Area",
    "Sub Area"
]

# Keep only columns that exist (protects against missing columns)
existing_cols = [c for c in required_cols if c in maint.columns]

fact_maintenance_event = maint[existing_cols].copy()

print("\nfact_maintenance_event sample:")
fact_maintenance_event.head()


fact_maintenance_event sample:


,start_datetime,duration_hours,duration_min,equipment_name,Crew,Shifts,Category,Delay Type,Area,Sub Area
3,2024-04-02 18:00:00,1.0,60.0,Temper Mill Unit,2,Christo,Planned downtime,Planned Downtime Production,Centre (2),Temper Mill Unit (2)
4,2024-04-23 10:50:00,1.0,60.0,Decoiler,4,David,Internal delays,Internal Maintenance,Entry (1),Decoiler (2)
5,2024-04-25 02:30:00,1.0,60.0,Operations,1,Gerhard,Internal delays,Internal Production,General (1),Operations (3)
6,2024-04-29 06:00:00,1.0,60.0,Common Equipment,1,Gerhard,External delays,External to Plant (S),General (1),Common Equipment(1)
7,2024-05-01 08:30:00,1.0,60.0,Entry SnubberHold Down & Pressure Rolls,4,David,Internal delays,Internal Maintenance,Entry (1),Entry SnubberHold Down & Pressure Rolls (4)


In [95]:
# ============================
# 7. BUILD 4-CREW 12-HOUR ROTATION
# ============================

unique_dates = sorted(fact_production_coil["production_date"].unique())
crew_codes = ["A", "B", "C", "D"]

# For each date: (day_crew, night_crew)
date_to_crews = {}
for idx, d in enumerate(unique_dates):
    day_crew = crew_codes[idx % 4]
    night_crew = crew_codes[(idx + 1) % 4]
    date_to_crews[d] = (day_crew, night_crew)

In [96]:
# ============================
# 8. REALISTIC DURATION & QUEUE LOGIC (WIDTH + THICKNESS AWARE)
# ============================

# Base operation duration ranges in SECONDS per coil, before product/shift/bottleneck adjustment.
# These are tuned so a "normal" coil ends up around 15 minutes for the full line.

DURATION_RANGES = {
    "Coil Prep Sattion": (30, 60),                     # 0.5–1.0 min
    "Decoiler": (60, 120),                             # 1–2 min (bottleneck)
    "Entry SnubberHold Down & Pressure Rolls": (20, 40),
    "Entry Guide Table": (10, 20),
    "Entry & Exit Feed Table": (20, 40),

    "Flattener, Pinch & Deflator Rolls": (40, 90),     # <1.5 min
    "Temper Mill Unit": (120, 240),                    # 2–4 min (primary bottleneck)

    "Crop Shear": (20, 40),

    "First Conveyor": (10, 20),
    "Second Conveyor": (10, 20),
    "Delivery Conveyor": (20, 40),

    "Scale M65 (conveyor)": (40, 90),                  # <1.5 min (bottleneck)
    "Exit Coil Car": (60, 120),                        # 1–2 min (bottleneck)
}

DEFAULT_RANGE = (20, 40)  # fallback for anything else


def get_duration_range(equipment_name: str):
    if equipment_name in DURATION_RANGES:
        return DURATION_RANGES[equipment_name]
    return DEFAULT_RANGE


# Shift multipliers to capture crew differences
SHIFT_MULTIPLIER = {
    "A": 1.05,   # slightly faster
    "B": 1.00,
    "C": 0.95,   # slightly slower
    "D": 1.00,
}


def product_mix_factor(thickness_mm: float, width_mm: float) -> float:
    """
    Product mix speed rule:
      - Narrow + thin coils run FASTER.
      - Wide > 3.5 mm and the rest of the mix sit around the baseline (~15 min/coil).

    We'll treat "narrow" as width <= 1300 mm (you can adjust)
    and "thin" as thickness <= 2.0 mm.

      - thin & narrow        -> 0.5–0.7  (much faster than baseline)
      - everything else      -> 0.9–1.1  (around 15 min baseline)
    """
    if pd.isna(thickness_mm) or pd.isna(width_mm):
        return 1.0

    t = float(thickness_mm)
    w = float(width_mm)

    is_thin = t <= 2.0
    is_narrow = w <= 1300  # adjust this cut-off if your line defines narrow differently

    if is_thin and is_narrow:
        # Fast-running coils
        return np.random.uniform(0.5, 0.7)
    else:
        # Wide/thicker and general product mix around 15 min
        return np.random.uniform(0.9, 1.1)


def draw_duration_seconds(equipment_name: str,
                          is_bottleneck: bool,
                          shift_code: str,
                          thickness_mm: float,
                          width_mm: float) -> float:
    """
    Draw a random operation duration (seconds) for a given equipment,
    adjusted for:
      - base equipment range
      - product mix (thickness + width)
      - shift performance
      - bottleneck behaviour
    """
    low, high = get_duration_range(equipment_name)
    base = np.random.uniform(low, high)

    mix_factor = product_mix_factor(thickness_mm, width_mm)
    shift_factor = SHIFT_MULTIPLIER.get(shift_code, 1.0)
    bottleneck_factor = 1.10 if is_bottleneck else 1.0

    return base * mix_factor * shift_factor * bottleneck_factor


def draw_queue_seconds(is_bottleneck: bool) -> float:
    """
    Queue time (seconds) before an equipment step.
    Bottlenecks experience slightly longer queues.
    """
    if is_bottleneck:
        return np.random.uniform(20, 60)    # 0.3–1.0 min
    return np.random.uniform(0, 20)         # 0–0.3 min

In [99]:
# ============================
# 9. GENERATE fact_coil_operation_cycle + SYNTHETIC COIL TIMES
# ============================

fact_coil_ops_rows = []
coil_time_info = {}   # coil_id -> (start_ts, end_ts, shift_code)

# Unique production dates
unique_dates = sorted(fact_production_coil["production_date"].unique())

for d in unique_dates:
    # Get day and night crews for this date
    day_crew, night_crew = date_to_crews[d]

    # Coils scheduled on this date
    day_coils = (
        fact_production_coil
        .loc[fact_production_coil["production_date"] == d]
        .sort_values("coil_id")
        .reset_index(drop=True)
    )
    if day_coils.empty:
        continue

    # Line starts at 06:00 on that date
    base_dt = datetime.combine(d, time(6, 0))
    current_line_ts = base_dt

    for _, coil in day_coils.iterrows():
        coil_id = coil["coil_id"]
        thickness_mm = coil["thickness_mm"]
        width_mm = coil["width_mm"]

        # Coil enters the line at current_line_ts
        coil_start_ts = current_line_ts

        # Decide which crew/shift runs this coil based on time of day
        if 6 <= coil_start_ts.hour < 18:
            shift_code = day_crew
        else:
            shift_code = night_crew

        ts = coil_start_ts

        # Walk the coil through each equipment in process order
        for _, eq in line_equipment.iterrows():
            equipment_id = eq["equipment_id"]
            equip_name = eq["equipment_name"]
            is_bottle = bool(eq["is_bottleneck_candidate"])

            # Queue before this equipment
            queue_sec = draw_queue_seconds(is_bottle)
            op_start_ts = ts + timedelta(seconds=queue_sec)

            # Operation duration at this equipment (now product-mix aware)
            dur_sec = draw_duration_seconds(
                equip_name,
                is_bottle,
                shift_code,
                thickness_mm,
                width_mm,
            )
            op_end_ts = op_start_ts + timedelta(seconds=dur_sec)

            # Record operation row
            fact_coil_ops_rows.append({
                "coil_id": coil_id,
                "equipment_id": equipment_id,
                "equipment_name": equip_name,
                "production_date": d,
                "shift_code": shift_code,
                "operation_start_ts": op_start_ts,
                "operation_end_ts": op_end_ts,
                "operation_duration_sec": dur_sec,
                "queue_time_sec": queue_sec,
                "is_bottleneck_step": is_bottle,
            })

            # Next equipment starts after this one ends
            ts = op_end_ts

        # Coil end time is after the last equipment
        coil_end_ts = ts
        coil_time_info[coil_id] = (coil_start_ts, coil_end_ts, shift_code)

        # Next coil starts when this one finishes
        current_line_ts = coil_end_ts

# Build fact_coil_operation_cycle DataFrame
fact_coil_operation_cycle = pd.DataFrame(fact_coil_ops_rows)

print("\nfact_coil_operation_cycle sample:")
fact_coil_operation_cycle.head()

# ============================
# 9.1 MERGE SYNTHETIC TIMES BACK TO fact_production_coil
# ============================

coil_times_df = (
    pd.DataFrame.from_dict(
        coil_time_info,
        orient="index",
        columns=["start_datetime", "end_datetime", "shift_code_synth"]
    )
    .reset_index()
    .rename(columns={"index": "coil_id"})
)

# Drop old synthetic columns if this section is rerun
cols_to_drop = [c for c in ["start_datetime", "end_datetime", "shift_code_synth"]
                if c in fact_production_coil.columns]
fact_production_coil = fact_production_coil.drop(columns=cols_to_drop)

# Merge new synthetic times
fact_production_coil = fact_production_coil.merge(
    coil_times_df,
    on="coil_id",
    how="left"
)

fact_production_coil["start_datetime"] = pd.to_datetime(fact_production_coil["start_datetime"])
fact_production_coil["end_datetime"] = pd.to_datetime(fact_production_coil["end_datetime"])

# Overwrite placeholder shift_code with synthetic crew assignment
fact_production_coil["shift_code"] = fact_production_coil["shift_code_synth"]
fact_production_coil.drop(columns=["shift_code_synth"], inplace=True)

# Total cycle time per coil in minutes
fact_production_coil["total_cycle_time_min"] = (
    (fact_production_coil["end_datetime"] - fact_production_coil["start_datetime"])
    .dt.total_seconds() / 60.0
)

print("\nUpdated fact_production_coil with synthetic times:")
fact_production_coil.head()


fact_coil_operation_cycle sample:

Updated fact_production_coil with synthetic times:


,coil_id,production_date,shift_code,thickness_mm,width_mm,mass_out_tons,Hours,Grade,NextProcess,total_cycle_time_min,start_datetime,end_datetime
0,7241079,2024-04-01,A,1.8,1211,23.40,00:38:51,AC1 37,13,12.413038,2024-04-01 17:00:32.250810,2024-04-01 17:12:57.033093
1,7255091,2024-04-01,B,2.3,1200,21.60,00:57:57,RC3 1,13,15.916516,2024-04-01 20:49:57.722302,2024-04-01 21:05:52.713247
2,7232058,2024-04-01,A,1.5,1236,22.56,00:25:08,AC2 31,13,12.411669,2024-04-01 12:06:01.641257,2024-04-01 12:18:26.341420
3,7232058,2024-04-01,A,1.5,1236,0.84,00:22:22,AC2 31,13,12.411669,2024-04-01 12:06:01.641257,2024-04-01 12:18:26.341420
4,7258061,2024-04-02,C,1.4,1232,21.94,00:18:53,AC2 31,13,10.374940,2024-04-03 02:56:40.267069,2024-04-03 03:07:02.763465


In [100]:
fact_coil_operation_cycle.head()

,coil_id,equipment_id,equipment_name,production_date,shift_code,operation_start_ts,operation_end_ts,operation_duration_sec,queue_time_sec,is_bottleneck_step
0,7111024,3,Coil Prep Sattion,2024-04-01,A,2024-04-01 06:00:11.300482,2024-04-01 06:00:52.555368,41.254886,11.300482,False
1,7111024,7,Decoiler,2024-04-01,A,2024-04-01 06:01:33.790335,2024-04-01 06:03:49.014010,135.223675,41.234967,True
2,7111024,11,Entry SnubberHold Down & Pressure Rolls,2024-04-01,A,2024-04-01 06:03:49.272309,2024-04-01 06:04:22.588907,33.316598,0.258299,False
3,7111024,10,Entry Guide Table,2024-04-01,A,2024-04-01 06:04:40.336644,2024-04-01 06:04:52.661140,12.324496,17.747737,False
4,7111024,9,Entry & Exit Feed Table,2024-04-01,A,2024-04-01 06:04:56.584769,2024-04-01 06:05:37.389809,40.805040,3.923629,False


In [103]:
# ============================
# 10. MERGE SYNTHETIC TIMES BACK TO fact_production_coil
# ============================

coil_times_df = (
    pd.DataFrame.from_dict(
        coil_time_info,
        orient="index",
        columns=["start_datetime", "end_datetime", "shift_code_synth"]
    )
    .reset_index()
    .rename(columns={"index": "coil_id"})
)

# Drop old synthetic columns if this block is rerun
cols_to_drop = [c for c in ["start_datetime", "end_datetime", "shift_code_synth"]
                if c in fact_production_coil.columns]
fact_production_coil = fact_production_coil.drop(columns=cols_to_drop)

# Merge synthetic times
fact_production_coil = fact_production_coil.merge(
    coil_times_df,
    on="coil_id",
    how="left"
)

# Ensure datetime types
fact_production_coil["start_datetime"] = pd.to_datetime(fact_production_coil["start_datetime"])
fact_production_coil["end_datetime"] = pd.to_datetime(fact_production_coil["end_datetime"])

# Overwrite placeholder shift_code with synthetic crew assignment
fact_production_coil["shift_code"] = fact_production_coil["shift_code_synth"]
fact_production_coil.drop(columns=["shift_code_synth"], inplace=True)

# Total cycle time per coil in minutes
fact_production_coil["total_cycle_time_min"] = (
    (fact_production_coil["end_datetime"] - fact_production_coil["start_datetime"])
    .dt.total_seconds() / 60.0
)

print("\nUpdated fact_production_coil with synthetic times:")
fact_production_coil


Updated fact_production_coil with synthetic times:


,coil_id,production_date,shift_code,thickness_mm,width_mm,mass_out_tons,Hours,Grade,NextProcess,total_cycle_time_min,start_datetime,end_datetime
0,7241079,2024-04-01,A,1.8,1211,23.40,00:38:51,AC1 37,13,12.413038,2024-04-01 17:00:32.250810,2024-04-01 17:12:57.033093
1,7255091,2024-04-01,B,2.3,1200,21.60,00:57:57,RC3 1,13,15.916516,2024-04-01 20:49:57.722302,2024-04-01 21:05:52.713247
2,7232058,2024-04-01,A,1.5,1236,22.56,00:25:08,AC2 31,13,12.411669,2024-04-01 12:06:01.641257,2024-04-01 12:18:26.341420
3,7232058,2024-04-01,A,1.5,1236,0.84,00:22:22,AC2 31,13,12.411669,2024-04-01 12:06:01.641257,2024-04-01 12:18:26.341420
4,7258061,2024-04-02,C,1.4,1232,21.94,00:18:53,AC2 31,13,10.374940,2024-04-03 02:56:40.267069,2024-04-03 03:07:02.763465
...,...,...,...,...,...,...,...,...,...,...,...,...
6485,7607111,2024-06-29,D,3.0,1362,21.24,23:56:17,AC1 37,13,16.712013,2024-06-29 09:20:49.945269,2024-06-29 09:37:32.666041
6486,7607113,2024-06-29,D,3.0,1358,1.50,23:48:05,AC1 37,13,15.216236,2024-06-29 09:52:24.496202,2024-06-29 10:07:37.470375
6487,7608020,2024-06-29,D,2.1,1513,0.56,23:32:52,AC2 32,13,16.987688,2024-06-29 11:26:04.757202,2024-06-29 11:43:04.018464
6488,7608031,2024-06-29,D,1.9,1517,1.56,23:02:05,AC2 32,13,16.457307,2024-06-29 13:50:03.542080,2024-06-29 14:06:30.980490


In [105]:
# ============================
# 10. BUILD fact_equipment_event_log (RUN / IDLE / FAULT)
# ============================

event_rows = []

# --- 10.1 RUN + IDLE from synthetic coil operations ---

# Use synthetic coil operations as the basis for RUN/IDLE
run_df = fact_coil_operation_cycle.copy()

# For convenience
run_df["event_start_ts"] = run_df["operation_start_ts"]
run_df["event_end_ts"] = run_df["operation_end_ts"]

# Add RUN and IDLE events per equipment
for eq_id, g in run_df.groupby("equipment_id"):
    g = g.sort_values("event_start_ts").reset_index(drop=True)
    equip_name = g["equipment_name"].iloc[0]

    # RUN events (one per coil at this equipment)
    for _, r in g.iterrows():
        event_rows.append({
            "equipment_id": eq_id,
            "equipment_name": equip_name,
            "event_type": "RUN",
            "event_start_ts": r["event_start_ts"],
            "event_end_ts": r["event_end_ts"],
            "event_duration_sec": (r["event_end_ts"] - r["event_start_ts"]).total_seconds(),
            "coil_id": r["coil_id"],
            "shift_code": r["shift_code"],
        })

    # IDLE events between RUNs (gaps)
    min_idle_sec = 30  # ignore tiny gaps (< 30 sec)

    for i in range(len(g) - 1):
        end_curr = g.loc[i, "event_end_ts"]
        start_next = g.loc[i + 1, "event_start_ts"]

        gap = (start_next - end_curr).total_seconds()
        if gap > min_idle_sec:
            event_rows.append({
                "equipment_id": eq_id,
                "equipment_name": equip_name,
                "event_type": "IDLE",
                "event_start_ts": end_curr,
                "event_end_ts": start_next,
                "event_duration_sec": gap,
                "coil_id": None,
                "shift_code": None,
            })

# --- 10.2 FAULT from maintenance events ---

# Map maintenance to equipment_id using cleaned equipment_name
faults = fact_maintenance_event.merge(
    dim_equipment[["equipment_id", "equipment_name"]],
    on="equipment_name",
    how="left"
)

# Compute FAULT end timestamps
faults["fault_start_ts"] = faults["start_datetime"]
faults["fault_end_ts"] = faults["start_datetime"] + pd.to_timedelta(
    faults["duration_min"], unit="m"
)

for _, row in faults.iterrows():
    eq_id = row["equipment_id"]
    equip_name = row["equipment_name"]

    # Skip faults that are not on modelled line equipment
    if pd.isna(eq_id):
        continue

    event_rows.append({
        "equipment_id": eq_id,
        "equipment_name": equip_name,
        "event_type": "FAULT",
        "event_start_ts": row["fault_start_ts"],
        "event_end_ts": row["fault_end_ts"],
        "event_duration_sec": (row["fault_end_ts"] - row["fault_start_ts"]).total_seconds(),
        "coil_id": None,
        "shift_code": row["Shifts"] if "Shifts" in faults.columns else None,
    })

# --- 10.3 Build fact_equipment_event_log DataFrame ---

fact_equipment_event_log = pd.DataFrame(event_rows)

# Derive event_date for aggregation by day
fact_equipment_event_log["event_date"] = fact_equipment_event_log["event_start_ts"].dt.date

print("\nfact_equipment_event_log sample:")
fact_equipment_event_log.head()


fact_equipment_event_log sample:


,equipment_id,equipment_name,event_type,event_start_ts,event_end_ts,event_duration_sec,coil_id,shift_code,event_date
0,3.0,Coil Prep Sattion,RUN,2024-04-01 06:00:11.300482,2024-04-01 06:00:52.555368,41.254886,7111024,A,2024-04-01
1,3.0,Coil Prep Sattion,RUN,2024-04-01 06:16:24.004285,2024-04-01 06:17:08.785217,44.780932,7111024,A,2024-04-01
2,3.0,Coil Prep Sattion,RUN,2024-04-01 06:30:55.855361,2024-04-01 06:31:29.262667,33.407306,7122038,A,2024-04-01
3,3.0,Coil Prep Sattion,RUN,2024-04-01 06:42:46.418159,2024-04-01 06:43:12.983507,26.565348,7131082,A,2024-04-01
4,3.0,Coil Prep Sattion,RUN,2024-04-01 06:56:29.702888,2024-04-01 06:56:56.763218,27.060330,7131082,A,2024-04-01


###VALIDATE OUR DATA

In [107]:
import numpy as np
import pandas as pd

# Quick sanity check
fact_production_coil[["thickness_mm", "width_mm", "total_cycle_time_min"]].describe().T

,count,mean,std,min,25%,50%,75%,max
thickness_mm,6490.0,2.161274,0.841929,1.400000,1.500000,1.900000,2.500000,6.000000
width_mm,6490.0,1238.745917,142.376036,787.000000,1224.000000,1230.000000,1237.000000,1859.000000
total_cycle_time_min,6490.0,13.553967,2.554180,8.797744,11.410539,12.367597,16.092413,20.532867


CONFIRM THE DOUBLE BAND

In [109]:
# Define product mix bands
def classify_product(row):
    t = row["thickness_mm"]
    w = row["width_mm"]
    if pd.isna(t) or pd.isna(w):
        return "unknown"

    is_thin = t <= 2.0
    is_narrow = w <= 1300  # adjust if needed

    if is_thin and is_narrow:
        return "thin & narrow (fast band)"
    else:
        return "other mix (~15min)"

prod_with_band = fact_production_coil.copy()
prod_with_band["product_band"] = prod_with_band.apply(classify_product, axis=1)

# Summary by product band
cycle_summary = (
    prod_with_band
    .groupby("product_band")["total_cycle_time_min"]
    .agg(["count", "mean", "std", "min", "max"])
    .sort_values("mean")
)

print("\nCycle time (minutes) by product band:")
print(cycle_summary)


Cycle time (minutes) by product band:
                           count       mean       std        min        max
product_band                                                               
thin & narrow (fast band)   3875  11.603161  0.796367   8.797744  14.214935
other mix (~15min)          2615  16.444742  1.119646  13.159633  20.532867


In [110]:
# Approximate coils per hour per band: 60 / mean(cycle time)
cycle_summary["approx_coils_per_hour"] = 60 / cycle_summary["mean"]
print("\nApproximate coils per hour by product band:")
print(cycle_summary[["mean", "approx_coils_per_hour"]])


Approximate coils per hour by product band:
                                mean  approx_coils_per_hour
product_band                                               
thin & narrow (fast band)  11.603161               5.171005
other mix (~15min)         16.444742               3.648583


In [111]:
# Global mean cycle time and implied coils/hour
mean_cycle = fact_production_coil["total_cycle_time_min"].mean()
coils_per_hour_global = 60 / mean_cycle

print(f"\nGlobal mean cycle time: {mean_cycle:.2f} min/coil")
print(f"Implied throughput: {coils_per_hour_global:.2f} coils/hour")


Global mean cycle time: 13.55 min/coil
Implied throughput: 4.43 coils/hour


EQUIPMENT BOTTLENECK VALIDATION

In [112]:
# Operation duration stats by equipment
equip_durations = (
    fact_coil_operation_cycle
    .groupby("equipment_name")["operation_duration_sec"]
    .agg(["count", "mean", "std", "min", "max"])
    .sort_values("mean", ascending=False)
)

equip_durations["mean_min"] = equip_durations["mean"] / 60.0

print("\nMean operation duration per equipment (sorted by longest):")
print(equip_durations[["count", "mean_min", "min", "max"]])


Mean operation duration per equipment (sorted by longest):
                                         count  mean_min        min  \
equipment_name                                                        
Temper Mill Unit                          6490  2.512153  63.749902   
Decoiler                                  6490  1.251118  32.906054   
Exit Coil Car                             6490  1.247024  32.868424   
Scale M65 (conveyor)                      6490  0.904457  21.483772   
Flattener, Pinch & Deflator Rolls         6490  0.824290  19.315103   
Coil Prep Sattion                         6490  0.570584  14.756529   
Delivery Conveyor                         6490  0.382316   9.845232   
Entry & Exit Feed Table                   6490  0.381927   9.875621   
Crop Shear                                6490  0.381322   9.689487   
Entry SnubberHold Down & Pressure Rolls   6490  0.379930   9.795097   
Second Conveyor                           6490  0.189902   4.799680   
Entry Guide Table

In [113]:
# Share of total operation time per equipment (bottleneck share)
total_op_time = fact_coil_operation_cycle["operation_duration_sec"].sum()

equip_share = (
    fact_coil_operation_cycle
    .groupby("equipment_name")["operation_duration_sec"]
    .sum()
    .to_frame("total_op_sec")
)

equip_share["share_of_line_%"] = 100 * equip_share["total_op_sec"] / total_op_time
equip_share["total_op_min"] = equip_share["total_op_sec"] / 60.0

equip_share = equip_share.sort_values("share_of_line_%", ascending=False)

print("\nShare of total operation time per equipment:")
print(equip_share)


Share of total operation time per equipment:
                                          total_op_sec  share_of_line_%  \
equipment_name                                                            
Temper Mill Unit                         978232.226273        26.712204   
Decoiler                                 487185.301972        13.303378   
Exit Coil Car                            485591.057545        13.259845   
Scale M65 (conveyor)                     352195.552303         9.617266   
Flattener, Pinch & Deflator Rolls        320978.331860         8.764829   
Coil Prep Sattion                        222185.388553         6.067129   
Delivery Conveyor                        148873.875114         4.065241   
Entry & Exit Feed Table                  148722.492951         4.061107   
Crop Shear                               148486.865853         4.054673   
Entry SnubberHold Down & Pressure Rolls  147944.743353         4.039869   
Second Conveyor                           73947.706254

In [114]:
# Queue statistics per equipment
queue_stats = (
    fact_coil_operation_cycle
    .groupby("equipment_name")["queue_time_sec"]
    .agg(["mean", "std", "min", "max"])
    .sort_values("mean", ascending=False)
)

queue_stats["mean_min"] = queue_stats["mean"] / 60.0

print("\nQueue time stats per equipment (sorted by longest mean queue):")
print(queue_stats[["mean_min", "min", "max"]])


Queue time stats per equipment (sorted by longest mean queue):
                                         mean_min        min        max
equipment_name                                                         
Temper Mill Unit                         0.665849  20.012283  59.995595
Exit Coil Car                            0.665716  20.015200  59.997140
Scale M65 (conveyor)                     0.665245  20.003407  59.997743
Decoiler                                 0.664183  20.007768  59.997346
Entry & Exit Feed Table                  0.168899   0.002484  19.998314
Second Conveyor                          0.167284   0.002000  19.995863
Entry SnubberHold Down & Pressure Rolls  0.167118   0.000696  19.997692
Coil Prep Sattion                        0.166888   0.000971  19.996894
Delivery Conveyor                        0.166754   0.000118  19.996065
Flattener, Pinch & Deflator Rolls        0.166580   0.003609  19.994464
First Conveyor                           0.166406   0.000512  19.993649


In [116]:
# Event time per equipment and type
equip_event_time = (
    fact_equipment_event_log
    .groupby(["equipment_name", "event_type"])["event_duration_sec"]
    .sum()
    .to_frame("total_sec")
    .reset_index()
)

equip_event_time["total_min"] = equip_event_time["total_sec"] / 60.0

# Pivot to RUN/IDLE/FAULT shares
equip_event_pivot = (
    equip_event_time
    .pivot_table(
        index="equipment_name",
        columns="event_type",
        values="total_min",
        aggfunc="sum",
        fill_value=0
    )
)

equip_event_pivot["total_min_all"] = equip_event_pivot.sum(axis=1)
for col in ["RUN", "IDLE", "FAULT"]:
    if col in equip_event_pivot.columns:
        equip_event_pivot[col + "_%"] = 100 * equip_event_pivot[col] / equip_event_pivot["total_min_all"]

print("\nRUN/IDLE/FAULT minutes & shares per equipment:")
equip_event_pivot


RUN/IDLE/FAULT minutes & shares per equipment:


event_type,FAULT,IDLE,RUN,total_min_all,RUN_%,IDLE_%,FAULT_%
equipment_name,,,,,,,
CCTV Camera,265.0,0.000000,0.000000,265.000000,0.000000,0.000000,100.000000
Central Hyd System,200.0,0.000000,0.000000,200.000000,0.000000,0.000000,100.000000
Coil Prep Sattion,75.0,127376.095171,3703.089809,131154.184980,2.823463,97.119352,0.057185
Common Equipment,350.0,0.000000,0.000000,350.000000,0.000000,0.000000,100.000000
Computer Room,20.0,0.000000,0.000000,20.000000,0.000000,0.000000,100.000000
Crop Shear,165.0,128594.619235,2474.781098,131234.400333,1.885772,97.988499,0.125729
Decoiler,387.0,123019.815602,8119.755032,131526.570634,6.173471,93.532292,0.294237
Delivery Conveyor,135.0,128591.835403,2481.231252,131208.066655,1.891066,98.006044,0.102890
Entry & Exit Feed Table,50.0,128594.086922,2478.708215,131122.795137,1.890372,98.071496,0.038132


###EXPORT FILES FOR BI ANALYSIS

In [117]:
import os

# ============================
# EXPORT DATA FOR POWER BI
# ============================

os.makedirs("data", exist_ok=True)

# 1. dim_equipment
dim_equipment.to_csv("data/dim_equipment.csv", index=False)

# 2. fact_production_coil
fact_production_coil.to_csv("data/fact_production_coil.csv", index=False)

# 3. fact_coil_operation_cycle
fact_coil_operation_cycle.to_csv("data/fact_coil_operation_cycle.csv", index=False)

# 4. fact_maintenance_event
fact_maintenance_event.to_csv("data/fact_maintenance_event.csv", index=False)

# 5. fact_equipment_event_log
fact_equipment_event_log.to_csv("data/fact_equipment_event_log.csv", index=False)

print("Export complete. Files written to ./data folder:")
print(os.listdir("data"))

Export complete. Files written to ./data folder:
['fact_coil_operation_cycle.csv', 'fact_maintenance_event.csv', 'dim_equipment.csv', 'fact_equipment_event_log.csv', 'fact_production_coil.csv']
